In [11]:
from PIL import Image
import cozmo
import cv2
from cozmo.util import degrees, distance_mm, speed_mmps
import Stitching
#import random
#import Localize
#import MCLocalize
#import MotionUpdate
#import Histogram
#import sys
import os

In [2]:
# File path configuration
# Only need to run this once
parent_dir = "/home/bendurham441/Documents/cs371-cozmo-fourthhour"
path = os.path.join(parent_dir, "pictures")
os.mkdir(path)

FileExistsError: [Errno 17] File exists: '/home/bendurham441/Documents/cs371-cozmo-fourthhour/pictures'

In [12]:
def cozmo_program(robot: cozmo.robot.Robot):
  robot.say_text("Okay Here we goooooooo").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True
  
  # Saves picture of what Cozmo sees every 10 degrees.
  degree = 0
  while(degree < 360) :
    fileName = "takingpics" + str(degree)
 
    robot.turn_in_place(degrees(10)).wait_for_completed()
    
    latest_image = robot.world.latest_image
    annotated = latest_image.annotate_image()
    if latest_image is not None:
      print("image = %s" % latest_image)
      converted = annotated.convert()
      file_path = os.path.join(path, fileName)
      converted.save(file_path, "JPEG", resolution=10)
    degree += 10

In [13]:
cozmo.run_program(cozmo_program)

/home/bendurham441/.local/lib/python3.8/site-packages/cozmo/event.py:488: DeprecationWarning: The loop argument is deprecated since Python 3.8, and scheduled for removal in Python 3.10.
  return await asyncio.wait_for(f, timeout, loop=self._loop)
2023-04-14 12:50:14,234 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2023-04-14 12:50:14,234 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2023-04-14 12:50:14,234 cozmo.general INFO     App connection established. sdk_version=1.4.10 cozmoclad_version=3.4.0 app_build_version=00003.00004.00000
2023-04-14 12:50:14,240 cozmo.general INFO     Found robot id=1
2023-04-14 12:50:14,240 cozmo.general INFO     Found robot id=1
2023-04-14 12:50:14,240 cozmo.general INFO     Found robot id=1
2023-04-14 12:50:14,254 cozmo.general INFO     Connected to Android device serial=89c6ff8d
2023-04-

image = <cozmo.world.CameraImage object at 0x7f1e69c7ed60>
image = <cozmo.world.CameraImage object at 0x7f1e6956a2b0>
image = <cozmo.world.CameraImage object at 0x7f1e4d8f13a0>
image = <cozmo.world.CameraImage object at 0x7f1e4d923460>
image = <cozmo.world.CameraImage object at 0x7f1e90259610>
image = <cozmo.world.CameraImage object at 0x7f1e695331c0>
image = <cozmo.world.CameraImage object at 0x7f1e69533e50>
image = <cozmo.world.CameraImage object at 0x7f1e6953d790>
image = <cozmo.world.CameraImage object at 0x7f1e6953d340>
image = <cozmo.world.CameraImage object at 0x7f1e4d9782b0>
image = <cozmo.world.CameraImage object at 0x7f1e69b9b130>
image = <cozmo.world.CameraImage object at 0x7f1e4d9782b0>
image = <cozmo.world.CameraImage object at 0x7f1e69c7ed60>
image = <cozmo.world.CameraImage object at 0x7f1e69b8f4c0>
image = <cozmo.world.CameraImage object at 0x7f1e69bacdc0>
image = <cozmo.world.CameraImage object at 0x7f1e4d978580>
image = <cozmo.world.CameraImage object at 0x7f1e6956aaf

2023-04-14 12:50:53,129 cozmo.general INFO     Shutting down connection
2023-04-14 12:50:53,129 cozmo.general INFO     Shutting down connection
2023-04-14 12:50:53,129 cozmo.general INFO     Shutting down connection
2023-04-14 12:50:53,133 cozmo.general INFO     Android serial=89c6ff8d disconnected.
2023-04-14 12:50:53,133 cozmo.general INFO     Android serial=89c6ff8d disconnected.
2023-04-14 12:50:53,133 cozmo.general INFO     Android serial=89c6ff8d disconnected.


image = <cozmo.world.CameraImage object at 0x7f1e69b8feb0>


In [8]:
def run():
  print("\n\n\n!!!!! STITCH !!! \n\n\n\n")
  images = []
  for i in range(36):
    file_path = os.path.join(path, 'takingpics' + str((i * 10)))
    images.append(cv2.imread(file_path))    
    #cv2.imwrite("picture" + str(i) + ".jpeg", images[i]) 
    print(i)  
  stitcher = cv2.Stitcher.create()
  print("got here")
  ret,pano = stitcher.stitch(images)
  print("test")
  cv2.imwrite('Panorama.jpeg',pano)

In [9]:
run()




!!!!! STITCH !!! 




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
got here
test
